In [ ]:
from initialize import *
from datasets import load_dataset

ds_or = load_dataset("bench-llm/or-bench", "or-bench-80k")
ds_or_hard = load_dataset("bench-llm/or-bench", "or-bench-hard-1k")
ds_or_toxic = load_dataset("bench-llm/or-bench", "or-bench-toxic")

In [ ]:
base_model_path = "meta-llama/Meta-Llama-3.1-8B-Instruct"
model_path = "cackerman/ft_0to31_interleaved_stdmixsafecombonarafa_none_mult0"#ft_randalias_0to31_interleaved_stdmixsafecombo6_none_mult0"
#model = load_model(model_path, base_model_path, bnb = False)
model.tokenizer.padding_side = "left"
model.eval()
sampling_kwargs = {"use_cache": True, "pad_token_id": model.tokenizer.eos_token_id, "max_new_tokens": 120, "do_sample": False, "top_p": None, "temperature": None}
seed = 42
N=10
df = ds_or_hard['train'].to_pandas()
sampled_df = df.groupby('category', group_keys=False).apply(lambda x: x.sample(n=N, random_state=seed))
categories = sampled_df['category'].tolist()
prompts = sampled_df['prompt'].tolist()
prompts = [f"<|start_header_id|>user<|end_header_id|>\n\n{prompt}<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n" for prompt in prompts]
#for category,prompt in zip(categories,prompts): print(f"{category}: {prompt}")
batch_size=8
generated_responses=[]
for batch_start in range(0, len(prompts), batch_size):
    print(f"Starting {batch_start}")
    batch_start_time = time.time()
    batch_end = min(batch_start + batch_size, len(prompts))
    current_batch = prompts[batch_start:batch_end]

    inputs = model.tokenizer(current_batch, return_tensors="pt", padding=True).to(model.device)

    with torch.no_grad():
        generated_output = model.generate(**inputs, **sampling_kwargs)

    generated_responses.extend(model.tokenizer.batch_decode(generated_output[:, inputs['input_ids'].size(1):], skip_special_tokens=True))

with open("gen_or_hard_ftwoc_narafa.jsonl", 'a', encoding='utf-8') as output_file:
    for b in range(len(generated_responses)):
        json.dump({"category": categories[b], "question": prompts[b], "response": generated_responses[b]}, output_file)
        output_file.write("\n")


In [ ]:
base_model_path = "meta-llama/Meta-Llama-3.1-8B-Instruct"
model_path = base_model_path#"cackerman/ft_randalias_0to31_interleaved_stdmixsafecombo6_none_mult0"#ft_0to31_interleaved_stdmixsafecombonarafa_none_mult0"#
#model = load_model(model_path, base_model_path, bnb = False)
model.tokenizer.padding_side = "left"
model.eval()
sampling_kwargs = {"use_cache": True, "pad_token_id": model.tokenizer.eos_token_id, "max_new_tokens": 120, "do_sample": False, "top_p": None, "temperature": None}
"""
harmful1_file_path = './refusal_cln.json'
harmful_responses, prompts = [], []
with open(harmful1_file_path, 'r') as file:
    data = json.load(file) 
    for item in data:
        prompts.append(item['question'])
        harmful_responses.append({'question': item['question'], 'answer': item['answer_harmful']})
"""
"""        
harmful2_file_path = './harmful_lat.json'
harmful_lat_responses, prompts = [], []
with open(harmful2_file_path, 'r') as file:
    data = json.load(file) 
    for item in data:
        prompts.append(item['question'])
        harmful_lat_responses.append({'question': item['question'], 'answer': item['answer_harmful']})
"""
"""
insults_file_path = './insults.jsonl'
mean_responses, prompts = [], []
with open(insults_file_path, 'r') as file:
    for line in file:
        d = json.loads(line)
        prompts.append(d['question'])
        mean_responses.append({'question': d['question'], 'answer': d['mean_answer']})
"""
harmful3_file_path = './trustai_msj.json'
harmful3_responses, prompts = [], []
with open(harmful3_file_path, 'r') as file:
    data = json.load(file) 
    for item in data:
        if item['category'].startswith("Harmful"): 
            prompts.append(item['user'])
            harmful3_responses.append({'category': item['category'], 'question': item['user'], 'answer': item['assistant']})
            
prompts = [f"<|start_header_id|>user<|end_header_id|>\n\n{prompt}<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n" for prompt in prompts]
batch_size=32
generated_responses=[]
for batch_start in range(0, len(prompts), batch_size):
    print(f"Starting {batch_start}")
    batch_start_time = time.time()
    batch_end = min(batch_start + batch_size, len(prompts))
    current_batch = prompts[batch_start:batch_end]

    inputs = model.tokenizer(current_batch, return_tensors="pt", padding=True).to(model.device)

    with torch.no_grad():
        generated_output = model.generate(**inputs, **sampling_kwargs)

    generated_responses.extend(model.tokenizer.batch_decode(generated_output[:, inputs['input_ids'].size(1):], skip_special_tokens=True))

data = [
    {
        "user": harmful3_responses[b]['question'],
        "assistant": harmful3_responses[b]['answer'],
        "assistant_harmless": generated_responses[b]
        ,"category": harmful3_responses[b]['category']

    }
    for b in range(len(generated_responses))
]

with open("trustai_msj_l38b.json", 'w', encoding='utf-8') as output_file:
    json.dump(data, output_file, indent=2)
"""

with open("insults_l38b.jsonl", 'a', encoding='utf-8') as output_file:
    for b in range(len(generated_responses)):
        json.dump({
            "question": prompts[b],
            "mean_answer": mean_responses[b]['answer'],
            "normal_answer": generated_responses[b]
        }, output_file)
        output_file.write('\n')
"""